In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_key="b36879e47f2d42109a3d54edb22f78c3",
    azure_endpoint="https://ecotribe.openai.azure.com/openai/deployments/Chat/chat/completions?api-version=2024-02-15-preview",
    api_version="2024-02-15-preview",
    model="gpt-4o-2024-08-06",
    temperature=0,
    max_tokens=None,
)

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import base64
from io import BytesIO

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

image_path="/home/akshay/Documents/conxai/Hacknow_SickAG/hacknow-ecotribe/apple_rotten.jpg"
image_base64 = image_to_base64(image_path)
prompt = ChatPromptTemplate.from_messages([
            ("system", '''You are an expert at detecting the type of fruits, and their qualities from an input image. These qualities include color, and surface texture. You also identify if there is any mould, and if the fruit is defective.
        The input images can include images of fresh, semi-ripen, fully-ripen, or rotten fruits.
        Your task is to return a json object with following keys:
        {{
        "object_name": str, # name of the fruit
        "color": str, # color of the fruit
        "surface_texture": str, # texture of the surface: ["smooth", "rough", "fuzzy", "wrinkled", "bumpy", "waxy", "spiky", "dimpled"] 
        "has_mould_spot": str, # yes or no
        "shape": str # options: ["normal", "distorted", "swollen", "shriveled"]
        }}
     
     These information would help the user to identify if the fruit is fresh, partially-ripen, fully-ripen, partially-rotten, or fully-roten.
     '''),
    ("human", "Please analyze this fruit image and return JSON object as mentioned in the instructions."),
    ("human", f"![fruit_image](data:image/jpeg;base64,{image_base64})")
])

output_parser = JsonOutputParser()

response = llm.invoke(prompt.format(), response_format={"type": "json_object"})

In [6]:
response

AIMessage(content='{\n    "object_name": "Apple",\n    "color": "red",\n    "surface_texture": "smooth",\n    "has_mould_spot": "no",\n    "shape": "normal"\n}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 14605, 'total_tokens': 14648, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': '

In [1]:
from backend.utils.workflow_manager import WorkflowManager
graph = WorkflowManager().returnGraph()

In [2]:
filepath = "/home/akshay/Documents/conxai/Hacknow_SickAG/hacknow-ecotribe/apple_rotten.jpg"

In [3]:
graph.invoke({"image_path": filepath})

{'image_analysis_result': {'object_name': 'Kiwi',
  'color': 'green',
  'surface_texture': 'fuzzy',
  'has_mould_spot': 'no',
  'shape': 'normal'},
 'hyperspectral_analysis_result': {'has_spoilage': 'yes',
  'description': 'The fruit appears to have significant external spoilage, with visible bruising and discoloration. The surface shows signs of rot, indicating that it is partially rotten.'},
 'gas_detector_result': {'level_of_ethylene': 'high',
  'level_of_CO2': 'normal',
  'level_of_methane': 'not_detected',
  'level_of_ammonia': 'not_detected',
  'level_of_hydrogen_sulfide': 'not_detected'},
 'ripen_assessment_result': {'has_ripend': 'yes',
  'state': 'full-spoiled',
  'description': 'The fruit shows significant external spoilage with visible bruising and discoloration, indicating it is partially rotten. The high level of ethylene suggests advanced spoilage.'},
 'defect_assessment_result': {'has_defect': 'yes',
  'state': ['bruising', 'rot'],
  'description': 'The fruit shows signi